# MAT330 - 3. Try a linear model
<span style="color:blue"> ** In this notebook, we train several models with a set of hyperparameters** </span>

<span style="color:red"> ** - What is the best model you achieve? ** </span>

<span style="color:red"> ** - Could you try other hyperparameters/other scikit models to do better? See the sklearn documentation on the web** </span>

In [ ]:
%matplotlib inline
import os
import numpy as np
from scipy import io
import matplotlib.pyplot as plt
import pandas as pd
from pandas.api.types import CategoricalDtype

## Load data

In [ ]:
#Specify the categorical features
categorical_features = ['basin','nature']


train_filename = 'https://raw.githubusercontent.com/brajard/MAT330-Practical-work/master/data/train.csv'

data_train = pd.read_csv(train_filename, dtype={cat: 'category' for cat in categorical_features})

# Extract the predictor (but not the target -> data leakage)
X = data_train.drop('target',axis=1, inplace=False)
y = data_train['target']

# Do the same with the test dataset
test_filename = 'https://raw.githubusercontent.com/brajard/MAT330-Practical-work/master/data/test.csv'

Xtest = pd.read_csv(test_filename, dtype={cat: 'category' for cat in categorical_features})


## Several models

### 1. Feature selection

In [ ]:
## Some exemple of function to compute new features

# compute the  max, mean and standard deviation of the wind inensity
def compute_wind(df):
    features_u = [name for name in df.columns if name.startswith('u_')]
    features_v = [name for name in df.columns if name.startswith('v_')]
    u = df.get(features_u)
    v = df.get(features_v)
    intensity = u.values**2 + v.values**2
    
    return df.assign(wind_mean = intensity.mean(axis=1),
                     wind_std = intensity.std(axis=1),
                    wind_max = intensity.max(axis=1))

# Compute the max, average and standard deviation of the pressure
def compute_zmean(df):
    features_z = [name for name in df.columns if name.startswith('z_')]
    z = df.get(features_z)
    return df.assign(z_mean = z.mean(axis=1),
                    z_std = z.std(axis=1),
                    z_max = z.max(axis=1))


In [ ]:
from sklearn.preprocessing import StandardScaler


#Feature processing (on image)
X = compute_wind(X)
X = compute_zmean(X)
Xtest = compute_wind(Xtest)
Xtest = compute_zmean(Xtest)

#Features to use
features = ['latitude', 'instant_t','longitude', 'windspeed',
       'hemisphere', 'Jday_predictor', 'initial_max_wind',
       'max_wind_change_12h','dist2land',
            'wind_mean','wind_std','wind_max',
            'z_mean','z_std','z_max']


Xin = X.get(features)
Xin_test = Xtest.get(features)



# Dummy encoding of categorical features
Xcat = pd.get_dummies(X[categorical_features],prefix=categorical_features)
Xcat_test = pd.get_dummies(Xtest[categorical_features],prefix=categorical_features)

# Concatenate encoded categorical feature to other features
Xin = pd.concat([Xin,Xcat],axis=1)
Xin_test = pd.concat([Xin_test,Xcat_test],axis=1)

# Equalization of the types:
Xin = Xin.astype(float)
Xin_test = Xin_test.astype(float)

print('Size of training set:',Xin.shape)
print('Size of test set:',Xin_test.shape)



### 2. Train into Val/Train

In [ ]:
from sklearn.utils import shuffle
np.random.seed(10)
ids = shuffle(X.stormid.unique())

#Take 80% for training
limit_train = int(.8*len(ids))

#Index of training/val
idx_train = X.index[X.stormid.isin(ids[:limit_train])]
idx_val = X.index[X.stormid.isin(ids[limit_train:])]

X_train, y_train = Xin.loc[idx_train], y.loc[idx_train]
X_val, y_val = Xin.loc[idx_val], y.loc[idx_val]



### 3. Standardization
For all features, transform your data such that mean=0 and std=1 (on the training data), and use the same parameters for transforming the test data also. 

In [ ]:
scaler = StandardScaler().fit(X_train)
X_train_scaled = scaler.transform(X_train)
X_val_scaled = scaler.transform(X_val)
X_test_scaled = scaler.transform(Xin_test)

### 4. Specify the models and hyperparameter grids
You can try other models and other hyperparameters

In [ ]:
from sklearn.linear_model import Lasso
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.model_selection import ParameterGrid


param_grid_lasso = {'alpha':[0,0.1,1,10]} #Note: alpha=0 is eq. to standard linear regression
param_grid_rf = {'n_estimators':[50,100,200], 'min_samples_split':[2,5], 'max_features':['auto','sqrt']}
param_grid_gb = {'n_estimators':[50,100,200], 'min_samples_split':[2,5], 'max_features':['auto','sqrt']}

### 5. "Grid-Search"

In [ ]:
# Lasso regression
lassoreg = Lasso()
params_lasso = pd.Series(ParameterGrid(param_grid_lasso))
score = pd.Series (np.nan,params_lasso.index)
for i in params_lasso.index:
    #Set the hyperparameters to the tested configuration
    lassoreg.set_params(**params_lasso.loc[i])
    
    #Train the model
    lassoreg.fit(X_train_scaled,y_train)
    
    #Compute the score
    score.loc[i] = lassoreg.score(X_val_scaled,y_val)
#Create a dataframe
lasso_gs = pd.DataFrame({'params':params_lasso,'score':score})
lasso_gs = lasso_gs.assign(regressor='lasso')
lasso_gs

In [ ]:
# RF regression
rfreg = RandomForestRegressor()
params_rf = pd.Series(ParameterGrid(param_grid_rf))
score = pd.Series (np.nan,params_rf.index)
for i in params_rf.index:
    
    #Set the hyperparameters to the tested configuration
    rfreg.set_params(**params_rf.loc[i])
    
    #Train the model
    rfreg.fit(X_train_scaled,y_train)
    
    #Compute the score
    score.loc[i] = rfreg.score(X_val_scaled,y_val)
    print(params_rf.loc[i],score.loc[i])
#Create a dataframe
rf_gs = pd.DataFrame({'params':params_rf,'score':score})
rf_gs = rf_gs.assign(regressor='rf')
rf_gs

In [ ]:
# RF regression
gbreg = GradientBoostingRegressor()
params_gb = pd.Series(ParameterGrid(param_grid_gb))
score = pd.Series (np.nan,params_gb.index)
for i in params_gb.index:
    
    #Set the hyperparameters to the tested configuration
    gbreg.set_params(**params_gb.loc[i])
    
    #Train the model
    gbreg.fit(X_train_scaled,y_train)
    
    #Compute the score
    score.loc[i] = gbreg.score(X_val_scaled,y_val)
    print(params_gb.loc[i],score.loc[i])
#Create a dataframe
gb_gs = pd.DataFrame({'params':params_gb,'score':score})
gb_gs = gb_gs.assign(regressor='gb')
gb_gs

In [ ]:
#Put all the results in the same dataframe
gs_summary = pd.concat([lasso_gs,rf_gs,gb_gs],axis=0, ignore_index=True)
gs_summary.sort_values('score',inplace=True,ascending=False)

#Top 3 best regressors
gs_summary.head(3)

### 6. Validate the best model

In [ ]:
#Define the best regressor
best = gs_summary.iloc[0]
if best.regressor == 'lasso':
    reg = Lasso(**best.params)
elif best.regressor == 'gb':
    reg = GradientBoostingRegressor(**best.params)
elif best.regressor == 'rf':
    reg = RandomForestRegressor(**best.params)
else:
    print('Regressor',best.regressor,'not known')

#Do one last fit
reg.fit(X_train_scaled,y_train)
y_val_predict = reg.predict(X_val_scaled)
plt.scatter(y_val,y_val_predict)
plt.plot([0,140],[0,140],'-k')
plt.show()

In [ ]:
score = reg.score(X_val_scaled,y_val)
print('linear regression score: {:.3f}'.format(score))

### 7. Predicting the test dataset

In [ ]:
#Change the name for the file (e.g. you last name, or a name for you algo)
name = 'best_regressor'

y_test_predict = reg.predict(X_test_scaled)
np.save('test_predict.' + name + '.npy',y_test_predict)

#Send the file by email to julien.brajard@nersc.no

In [ ]:
X.columns.values[:30]